In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), '../'))

from evaluation.evaluation_harness import OnlineEvaluator, include_coldstart, include_all, score_time_spent, score_constant
from statistical_test import estimate_probability_each_model_is_best
from pprint import pprint
import pandas as pd
sys.path.append(os.path.join(os.path.abspath(''), '../backend'))
from backend.blueprints.recommendation import model_wrappers

%load_ext autoreload
%autoreload 2

In [2]:
evaluator = OnlineEvaluator(include_all, score_constant)
all_metrics = []
def run_eval(model_name, model_save_path):
    evaluator.reset(model_name, model_save_path)
    # evaluator.plot_top_N_hit_percentage_percentiles(10)
    evaluator.compute_top_N_hit_percentage(10)
    # evaluator.plot_top_N_hit_percentage_percentiles(50)
    evaluator.compute_top_N_hit_percentage(50)
    # evaluator.plot_user_rank_roc_curve()
    evaluator.compute_user_rank_auc_roc()
    # evaluator.save_metrics('test_online_evaluator', overwrite=True)
    # pprint(evaluator.metrics)
    all_metrics.append({'model_name': model_name, 'model_save_path': model_save_path, **evaluator.metrics})
    # print()
evaluator.all_results.sort_values('timestamp', inplace=True)

In [3]:
current_models = []
for model_wrapper in model_wrappers:
    model_wrapper.model = model_wrapper.definition()
    model_wrapper.model.load(model_wrapper.model_save_file_name, load_published_model=True)
    current_models.append((model_wrapper.model.name(), model_wrapper.model_save_file_name))
# groups = evaluator.all_results.groupby(by=["rec_model_name", "rec_model_save_path"])
# for (model_name, model_save_path), data in groups:
all_metrics = []
for model_name, model_save_path in current_models:
    run_eval(model_name, model_save_path)
results = pd.DataFrame(all_metrics)

In [4]:
estimate_probability_each_model_is_best(results, 'user_rank_auc_roc')
estimate_probability_each_model_is_best(results, 'top_10_hit_percentage')
estimate_probability_each_model_is_best(results, 'top_50_hit_percentage')

In [7]:
evaluation_names = {'evaluation_test_random_model': 'random_model', 'evaluation_test_popularity_model': 'popularity_model', 'evaluation_test_common_neighbors_constant_scoring': 'common_neighbors_constant_scoring', 'evaluation_test_common_neighbors': 'common_neighbors_gaussian_scoring', 'evaluation_test_common_neighbors_percentile_scoring': 'common_neighbors_percentile_scoring', 'evaluation_test_cf_low_weight_decay_increased_lr': 'collborative_filter', 'evaluation_test_gcf_low_weight_decay_increased_lr': 'non_linear_collaborative_filter', 'evaluation_test_mlp_low_weight_decay_increased_lr': 'multilayer_perceptron', 'evaluation_test_ncf_low_weight_decay_increased_lr': 'neural_collaborative_filter', 'evaluation_test_cf_embed_all_except_tags_genres': 'collborative_filter_with_game_embeddings', 'evaluation_test_gcf_embed_all_except_tags_genres': 'non_linear_collaborative_filter_with_game_embeddings', 'evaluation_test_mlp_embed_all_except_tags_genres': 'multilayer_perceptron_with_game_embeddings', 'evaluation_test_ncf_embed_all_except_tags_genres': 'neural_collaborative_filter_with_game_embeddings'}
df = results.loc[results['model_save_path'].isin(evaluation_names), ['model_save_path', 'top_10_hit_percentage', 'top_10_hit_percentage_variance', 'top_10_hit_percentage_best_probability', 'top_50_hit_percentage', 'top_50_hit_percentage_variance', 'top_50_hit_percentage_best_probability', 'user_rank_auc_roc', 'user_rank_auc_roc_variance', 'user_rank_auc_roc_best_probability']].copy()
df['model_save_path'] = df['model_save_path'].map(evaluation_names)
display(df.sort_values(by='top_50_hit_percentage_best_probability', ascending=False))

,model_save_path,top_10_hit_percentage,top_10_hit_percentage_variance,top_10_hit_percentage_best_probability,top_50_hit_percentage,top_50_hit_percentage_variance,top_50_hit_percentage_best_probability,user_rank_auc_roc,user_rank_auc_roc_variance,user_rank_auc_roc_best_probability
2,common_neighbors_gaussian_scoring,0.592593,0.019943,0.35605,0.573770,0.015677,0.38670,0.555495,0.005668,0.00000
1,common_neighbors_percentile_scoring,0.407407,0.037037,0.09615,0.480000,0.033265,0.21690,0.368590,0.006364,0.00000
0,common_neighbors_constant_scoring,0.350000,0.040643,0.06414,0.411765,0.035780,0.12751,0.485714,0.012676,0.00006
8,neural_collaborative_filter,0.407407,0.079772,0.17573,0.323529,0.063153,0.10558,0.498024,0.009623,0.00000
6,non_linear_collaborative_filter,0.388889,0.081699,0.16113,0.321429,0.044709,0.07143,0.529240,0.015720,0.00075
4,random_model,0.333333,0.076923,0.11119,0.333333,0.036444,0.06185,0.503827,0.009416,0.00001
7,multilayer_perceptron,0.111111,0.000000,0.00000,0.333333,0.017021,0.02034,0.227539,0.005650,0.00000
3,popularity_model,0.222222,0.034188,0.01315,0.373134,0.003408,0.00454,0.312857,0.005285,0.00000
11,multilayer_perceptron_with_game_embeddings,0.153846,0.057692,0.01816,0.086957,0.037549,0.00408,0.952381,0.001942,0.95974
5,collborative_filter,0.375000,0.002381,0.00312,0.366667,0.001207,0.00100,0.464115,0.011727,0.00004
